# Dataloader

# 0. arguments in dataloader

dataloader의 arguments는 다음과 같이 구성됩니다.

```python
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)
```

여기에서는 

* dataset
* num_workers and pin_memory
* collate_fn

에 대해서 알아봅니다.

다른 사항 혹은 더 자세한 내용은 [여기](https://pytorch.org/docs/stable/data.html)를 참고해주세요.

## 0. map-style dataset

dataset에는

* map-style과
* iterable-style의 dataset이 존재합니다.

Map-style dataset은 dataset class안에 ```__getitem__``` 함수와 ```__len__``` 함수가 존재합니다. (이를 python slice obejct라고 합니다.)

In [1]:
from torch.utils.data import Dataset
import numpy as np

class map_dataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.data = np.arange(5.)
    
    def __getitem__(self, i):
        # slicing rule with index i
        d = self.data[i]
        return  d
    
    def __len__(self):
        return len(self.data)
    
m_dataset = map_dataset()

여기에서 ```__getitem__``` 함수는 다음과 같이 두 가지 방식으로 사용할 수 있습니다.

In [2]:
print(m_dataset.__getitem__(2))
print(m_dataset[2])

2.0
2.0


마찬가지로 ```__len__``` 함수는 다음과 같이 사용할 수 있습니다.

In [3]:
print(m_dataset.__len__())
print(len(m_dataset))

5
5


따라서 우리는 기존의 python slicing에 사용하던 다음 문법을 사용할 수 있습니다.

In [4]:
print(m_dataset[0:4:2])
print(m_dataset[-1])

[0. 2.]
4.0


## 1. iterable-style dataset

Iterable-style dataset은 dataset class안에 ```__iter__``` 함수가 존재합니다. (이를 python iterable object라고 합니다.)

In [5]:
class iter_dataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.data = np.arange(5.)
    
    def __iter__(self):
        return None
    
i_dataset = iter_dataset()
from collections.abc import Iterable
print(isinstance(i_dataset, Iterable))

True


ML에서는 보통 iterable 객체가 아니라 여기에 ```next``` 함수를 적용할 수 있는 iterator를 사용합니다.

In [6]:
class iter_dataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.data = np.arange(5.)
        self.i = 0
        self.n = len(self.data)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < self.n:
            return self.data[self.i]
        else:
            raise StopIteration()
            
    # this function is optional!
    def __len__(self):
        return self.n
    
i_dataset = iter_dataset()
iter(i_dataset)
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))

# print(next(i_dataset))
# IndexError: index 5 is out of bounds for axis 0 with size 5

1.0
2.0
3.0
4.0


이렇게 정의된 iterator의 전형적인 사용 방식은 다음과 같습니다.

In [7]:
for d in iter_dataset():
    print(d)

for i, d in enumerate(iter_dataset()):
    print(i, d)

1.0
2.0
3.0
4.0
0 1.0
1 2.0
2 3.0
3 4.0


한편 ```__len__```이 구현되어 있는 함수의 경우에는 progress bar를 다음과 같이 볼 수 있습니다.

이때 progress bar와 함께 볼 수 있는 it/s 정보는 num_workers를 결정할때 유용합니다.

In [8]:
from tqdm import tqdm_notebook
for i,d in tqdm_notebook(enumerate(iter_dataset())):
    print(i,d)
    
from tqdm import tqdm
for i,d in tqdm(enumerate(iter_dataset())):
    print(i,d)

4it [00:00, 13923.00it/s]

0 1.0
1 2.0
2 3.0
3 4.0

0 1.0
1 2.0
2 3.0
3 4.0


마지막으로 길이가 정해져 있지 않는 경우에도 다음과 같이 구현할 수 있습니다.

In [9]:
class iter_dataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.i = 0
    
    # this function is a generator
    def __iter__(self):
        while True:
            self.i += 1
            yield self.i
    
i_dataset = iter(iter_dataset())
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))
print(next(i_dataset))

1
2
3
4
5
6


위에서 Dataloader가 argument로 받을 수 있었던 sampler는 ```next``` 함수가 없는 map-style dataset에서 data loading 순서를 정하고 싶을때 유용하게 사용할 수 있습니다.

## 2. num_workers

Dataloader argument에서 default num_workers 값은 0으로 이는 main process를 사용해 data loading을 한다는 뜻입니다.

그밖에도 양의 정수값을 num_workers로 설정할 수 있지만, 어떤 값이 가장 좋은지는 

* 알고리즘, 
* 데이터, 
* 구동 사양에 

따라 다르기 때문에 여러값을 해보시는 것을 추천드립니다.

## 3. pin_memory

```pin_memory``` 함수는 cpu에 있는 tensor를 받아서 gpu에 빠르게 올릴 수 있도록 data를 저장합니다. 

Dataloader에 있는 argument인 pin_memory는 이 pin_memory 함수를 data loading하는 과정에서 자동으로 부를 지를 선택할 수 있습니다.

한 가지 유의해야할 점은 이렇게 제공되는 pin_memory argument는 mini batch가 tensor 혹은 tensor를 포함한 iterable일때만 제대로 작동한다는 점입니다. 그렇지 않은 경우에는 [여기](https://pytorch.org/docs/stable/data.html#memory-pinning)를 참고해주세요.

## 4. collate_fn

```collate_fn``` 의 default값은 ```None```으로 이 경우에 ```collate_fn```이 하는 역할은

* ```(data, label)```의 tuple의 list로 구성되어 있는 mini-batch를 ```(mini-batch data, mini-batc label)```의 single tuple로 바꿔줍니다.(따라서 mini-batch data는 기존의 data보다 차원이 하나 더 추가됩니다.)
* 이 과정에서 numerical value (python, numpy)들을 tensor로 바꿔줍니다.

입니다.

한편, 이 과정을 customize하는 것 또한 가능한데 이를 위해서는 직접 ```list```를 받아서 비슷한 역할을 해줄 수 있는 함수를 작성해야 합니다.

In [12]:
# thanks to Juhyuk Lee (https://github.com/sehkmg)
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class map_dataset(Dataset):
    
    def __init__(self):
        super().__init__()
        self.data = np.arange(50.)
    
    def __getitem__(self, i):
        # slicing rule with index i
        d = self.data[i]
        return  d
    
    def __len__(self):
        return len(self.data)
    
m_dataset = map_dataset()

def negative_fn(batch):
    batch = [-b for b in batch]
#     batch = torch.Tensor(batch)
    return batch

m_dataloader = DataLoader(m_dataset, batch_size=10, shuffle=True)
m_dataloader_neg = DataLoader(m_dataset, batch_size=10, shuffle=True, collate_fn=negative_fn)

print('Basic dataloader:')
for batch in m_dataloader:
    print(batch)

print('Dataloader with negative_fn:')
for batch in m_dataloader_neg:
    print(batch)

Basic dataloader:
tensor([34.,  0., 15., 16.,  2., 24.,  3., 30., 21., 19.], dtype=torch.float64)
tensor([27.,  6., 48., 38., 35., 10., 45., 39.,  8., 11.], dtype=torch.float64)
tensor([33., 22., 42.,  4., 37., 13.,  7., 49., 41., 12.], dtype=torch.float64)
tensor([46., 32., 40., 31., 47., 25., 14., 17.,  1., 43.], dtype=torch.float64)
tensor([26., 20.,  5., 29., 28., 44., 18., 36., 23.,  9.], dtype=torch.float64)
Dataloader with negative_fn:
[-3.0, -21.0, -33.0, -6.0, -16.0, -37.0, -27.0, -29.0, -5.0, -19.0]
[-32.0, -12.0, -47.0, -4.0, -30.0, -28.0, -24.0, -11.0, -43.0, -1.0]
[-20.0, -9.0, -15.0, -42.0, -2.0, -39.0, -48.0, -38.0, -44.0, -41.0]
[-10.0, -40.0, -46.0, -35.0, -17.0, -22.0, -0.0, -45.0, -36.0, -34.0]
[-7.0, -8.0, -14.0, -23.0, -26.0, -25.0, -18.0, -13.0, -49.0, -31.0]


## 5. random seed fix

실험의 reproducibility를 위해서는 pseudo random number generator의 seed를 고정하여 randomness가 개입되는 상황 (mini-batch sampling, random sampling with fixed distribution)에서 같은 숫자가 sample되도록 하는 것이 중요합니다. 아래의 함수는 이와 같은 상황에서 유용합니다. (하지만 주석의 링크에서 확인할 수 있듯이, 이렇게 해도 완전한 reproducing을 보장하지는 못합니다.)

In [11]:
def set_seed(seed):
    import os
    import random
    import numpy as np
    import torch
    # for reproducibility. 
    # note that pytorch is not completely reproducible 
    # https://pytorch.org/docs/stable/notes/randomness.html  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.initial_seed() # dataloader multi processing 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return None